In [1]:
# /// script
# requires-python = ">=3.11"
# dependencies = [
#     "duckdb==1.4.3",
#     "h3>=4.0.0",
#     "lonboard==0.13.0",
#     "matplotlib==3.10.8",
#     "morecantile>=1.0.0",
#     "numpy==2.2.0",
#     "odc-stac==0.5.0",
#     "palettable==3.3.3",
#     "planetary-computer==1.0.0",
#     "pyarrow==18.1.0",
#     "pyproj==3.7.2",
#     "pystac-client==0.9.0"
# ]
# ///

In [2]:
import numpy as np
import pyarrow as pa
import duckdb
import morecantile
import h3
import odc.stac
import planetary_computer
import pystac_client
from concurrent.futures import ThreadPoolExecutor, as_completed
from matplotlib.colors import Normalize
from palettable.scientific.sequential import LaJolla_20, LaJolla_20_r, Bamako_20
from palettable.matplotlib import Viridis_20, Viridis_20_r
from pyproj import Transformer
from arro3.core import Table

from lonboard import Map, H3HexagonLayer
from lonboard.colormap import apply_continuous_cmap

import warnings                                                                                                                                                                    
warnings.filterwarnings("ignore", message="Dataset has no geotransform", category=UserWarning)

## Pipeline Functions

In [3]:
def calculate_resolution_for_h3(h3_res, native_resolution=10, pixels_per_hex_edge=6):
    """Calculate odc-stac resolution to get ~pixels_per_hex_edge pixels per H3 hex edge.

    For COGs, resolution controls which internal overview GDAL reads:
    coarser resolution = smaller overview = less I/O.
    """
    hex_edge_m = h3.average_hexagon_edge_length(h3_res, unit='m')
    target = hex_edge_m / pixels_per_hex_edge
    resolution = max(round(target / native_resolution) * native_resolution, native_resolution)
    px_per_edge = hex_edge_m / resolution
    print(f"H3 res {h3_res}: hex edge {hex_edge_m:.0f}m, resolution {resolution}m, {px_per_edge:.1f} px/edge")
    return resolution


def query_stac(bbox, collection):
    """Query Planetary Computer STAC catalog for items covering bbox."""
    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1",
        modifier=planetary_computer.sign_inplace,
    )
    items = catalog.search(
        collections=[collection],
        bbox=bbox,
        query={"gsd": {"eq": 10}} 
    ).item_collection()
    print(f"Found {len(items)} STAC items")
    return items


def get_tiles(bbox, zoom):
    """Split bbox into morecantile tiles at given zoom level."""
    tms = morecantile.tms.get("WebMercatorQuad")
    tiles = list(tms.tiles(*bbox, zooms=[zoom]))
    print(f"{len(tiles)} tiles at zoom {zoom}")
    return tiles, tms


# Install extensions once globally
duckdb.sql("INSTALL h3 FROM community")


def get_con():
    """In-memory connection for workers. LOAD only, no INSTALL."""
    con = duckdb.connect()
    con.sql("""
        SET temp_directory = './tmp';
        SET memory_limit = '512MB';
        LOAD h3;
""")
    return con


def process_tile_to_h3(tile, tms, items, band, h3_res, resolution):
    """Load one tile's DEM, reproject to 4326, aggregate to H3 immediately.

    Returns Arrow table (hex, metric) or None on failure.
    """
    tile_bounds = tms.bounds(tile)
    tile_bbox = [tile_bounds.left, tile_bounds.bottom, tile_bounds.right, tile_bounds.top]
    transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

    try:
        ds = odc.stac.load(
            items,
            crs="EPSG:3857",
            resolution=resolution,
            bands=[band],
            bbox=tile_bbox,
        ).astype(float)
    except Exception:
        return None

    arr = ds[band].max(dim="time")
    vals = arr.values
    x_coords = arr.coords["x"].values
    y_coords = arr.coords["y"].values
    X, Y = np.meshgrid(x_coords, y_coords)
    lons, lats = transformer.transform(X.flatten(), Y.flatten())

    tile_pa = pa.table({
        "lat": pa.array(lats, type=pa.float64()),
        "lng": pa.array(lons, type=pa.float64()),
        "elevation": pa.array(vals.flatten(), type=pa.float64()),
    })

    con = get_con()
    result = con.sql(f"""
        SELECT
            h3_latlng_to_cell_string(lat, lng, {h3_res}) AS hex,
            AVG(elevation) AS metric
        FROM tile_pa
        GROUP BY 1
    """).fetch_arrow_table()
    return result


def process_all_tiles(items, tiles, tms, band, h3_res, resolution, max_workers=4):
    """Process all tiles concurrently with per-tile H3 aggregation, then merge edge hexagons."""
    batches = []
    completed = 0
    total = len(tiles)

    with ThreadPoolExecutor(max_workers=max_workers) as pool:
        futures = {
            pool.submit(process_tile_to_h3, tile, tms, items, band, h3_res, resolution): tile
            for tile in tiles
        }
        for future in as_completed(futures):
            result = future.result()
            if result is not None and len(result) > 0:
                batches.append(result)
            completed += 1
            if completed % 100 == 0 or completed == total:
                print(f"  Processed {completed}/{total} tiles")

    if not batches:
        raise RuntimeError("No tiles produced data")

    combined = pa.concat_tables(batches)
    print(f"Pre-merge hex count: {len(combined):,}")

    # Final merge: hexagons on tile edges appear in multiple batches
    con = duckdb.connect()
    hex_result = con.sql("""
        SELECT hex, AVG(metric) AS metric
        FROM combined
        GROUP BY 1
    """).fetch_arrow_table()
    con.close()
    print(f"Final H3 hexagons: {len(hex_result):,}")
    return hex_result

## Configuration

In [4]:
# Grand Canyon bbox: west, south, east, north
# bbox = [-112.23047, 35.926336, -111.749034, 36.268266]
#bigger grand canyon bbox:
bbox =[-113.0606,35.8461,-111.7165,36.7665]
#yazoo city MS
# bbox = [-90.9642,32.6594,-90.109,33.477] # missed it
# cairo IL
# bbox = [-89.4436,36.91,-89.0463,37.1834]
# greenville MS
# bbox = [-91.3756,32.447,-90.859,33.9034]
# greater mount washington nh
# bbox = [-71.502182,44.092909,-70.723358,44.511611]
# bigger mount wash
# bbox= [-71.7445,43.8609,-70.602,44.7055]
COLLECTION = "3dep-seamless"
BAND = "data"
H3_RES = 11
NATIVE_RESOLUTION = 10  # 3DEP seamless native pixel size (meters)
RESOLUTION = calculate_resolution_for_h3(H3_RES, NATIVE_RESOLUTION)
TILE_ZOOM = 12
MAX_WORKERS = 8

H3 res 11: hex edge 29m, resolution 10m, 2.9 px/edge


## Run Pipeline

In [ ]:
items = query_stac(bbox, COLLECTION)
tiles, tms = get_tiles(bbox, TILE_ZOOM)
hex_result = process_all_tiles(items, tiles, tms, BAND, H3_RES, RESOLUTION, max_workers=MAX_WORKERS)

Found 6 STAC items
224 tiles at zoom 12


## Visualize

In [ ]:
# # def build_map(hex_result, bbox, colormap=LaJolla_20_r, view_state=None):
# # from palettable.matplotlib import Viridis_20_r

# """Build lonboard Map with extruded H3HexagonLayer."""
# elev_values = hex_result["metric"].to_pylist()
# elev_min = min(elev_values)
# elev_max = max(elev_values)
# colormap=Viridis_20
                                                                                                                                             
# # from matplotlib.colors import BoundaryNorm 
# # boundaries = [23, 30, 34]                                                                                                               
#   # evenly spaced across [0, 1]                                                                                                            
# # normalized = np.interp(elev_values, boundaries, np.linspace(0, 1, len(boundaries)))     
# normalizer = Normalize(elev_min , elev_max)
# normalized = normalizer(elev_values)
# colors = apply_continuous_cmap(normalized, colormap, alpha=1)

table = Table.from_arrow(hex_result)
# del hex_result
# layer = H3HexagonLayer(
#     table=table,
#     get_hexagon=table["hex"],
#     get_fill_color=colors,
#     high_precision=True, 
#     stroked=False,
#     get_elevation=table["metric"],
#     extruded=True,
#     elevation_scale=3.4,
#     opacity=0.7,
# )

# # if view_state is None:
# view_state = {
#     "longitude": (bbox[0] + bbox[2]) / 2,
#     "latitude": (bbox[1] + bbox[3]) / 2,
#     "zoom": 12,
#     "pitch": 55,
#     "bearing": -20,
# }


In [9]:
# def build_map(hex_result, bbox, colormap=LaJolla_20_r, view_state=None):
# from palettable.matplotlib import Viridis_20_r

"""Build lonboard Map with extruded H3HexagonLayer."""
elev_values = table["metric"].to_pylist()
elev_min = min(elev_values)
elev_max = max(elev_values)
colormap=Viridis_20_r
                                                                                                                                             
# from matplotlib.colors import BoundaryNorm 
# boundaries = [23, 30, 34]                                                                                                               
  # evenly spaced across [0, 1]                                                                                                            
# normalized = np.interp(elev_values, boundaries, np.linspace(0, 1, len(boundaries)))     
normalizer = Normalize(elev_min , elev_max)
normalized = normalizer(elev_values)
colors = apply_continuous_cmap(normalized, colormap, alpha=1)

# table = Table.from_arrow(hex_result)
# del hex_result
layer = H3HexagonLayer(
    table=table,
    get_hexagon=table["hex"],
    get_fill_color=colors,
    high_precision=True, 
    stroked=False,
    get_elevation=table["metric"],
    extruded=True,
    elevation_scale=3.4,
    opacity=0.9,
)

# if view_state is None:
view_state = {
    "longitude": (bbox[0] + bbox[2]) / 2,
    "latitude": (bbox[1] + bbox[3]) / 2,
    "zoom": 12,
    "pitch": 55,
    "bearing": -20,
}


In [10]:
# from palettable.scientific.sequential import LaJolla_20, Bamako_20
m = Map(layers=[layer], view_state=view_state)
m